# 项目：实时时钟显示

用Qt制作一个能显示时间的应用程序

# Step0. [pytb安装]

如果已安装，请直接跳过，否则命令行输入如下命令：  
```conda install pytb```

# Step1. 创建主窗口

In [1]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-

import sys
from PyQt5.QtWidgets import QApplication, QWidget

import导入sys、QApplication、QWidget模块。等价于C语言中的`#include`. 

In [2]:
app = QApplication(sys.argv)

调用函数QApplication用于创建一个Qt应用程序

In [3]:
w = QWidget()
w.show()

使用QWidget创建一个窗体w，并通过w.show方法把窗体显示出来

In [4]:
app.exec_()

0

exec()函数有一个消息循环，在调用之后，程序就被锁定。等待窗口的关闭

## 修改窗口名称

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget
app = QApplication(sys.argv)
w = QWidget()
w.show()

In [ ]:
w.setWindowTitle("Hello")

修改QWidget对象w的WindowTitle属性为“Hello”

In [ ]:
app.exec_()

## C++版本
```
// main.cpp
#include <QApplication>
#include <QWidget>

int main(int argc, char *argv[])
{
    QApplication app(argc, argv);
    
    //创建主窗口
    QWidget w;
    //修改主窗口名称为"Hello"
    w.setWindowsTitle("Hello");
    //显示主窗口
    w.show();
    
    return app.exec(); // 程序锁定，等待关闭
}
```
```
# test.pro
QT       += core gui widgets

SOURCES += \
        main.cpp
```

# Step2. 窗体上显示文字-标签使用

In [ ]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-

import sys
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QLabel

def main():
    QApplication app(argc, argv);
        
    w = QWidget()
    w.show()
    
    text = QLabel()
    text.setText("Hello World")
    text.show()
    
    app.exec()  # 锁定程序，等待退出
    
if __name__ == '__main__':
    main()

* 用`def main():`将多行命令代码封装为函数。
* 导入QLabel模块并创建对象。  
* 执行后会发现输出为2个窗口，一个为Hello World,一个为Python。如何让文本显示在主窗体上呢

其实QLabel与QWidget是有继承关系的，QLable继承自QFrame，QFrame又继承自QWidget。QWidget是主窗口，如果想让QLabel显示在QWidget创建的主窗口中，就需要将Qlabel对象text的父对象设为w。
```
text = QLabel()
text.setText("Hello World")
text.show()
text.setParent(w)
```

## C++版本

```
#include <QLabel>

...

QWidget w;
w.show();

text = QLabel() //创建标签
text.setText("Hello World") //修改标签文本属性
text.show() //标签显示
```

使用text对象的setParent方法绑定父对象

`text.setParent(&w)`C++中需要使用指针作为函数的入口参数

## 小任务
修改代码使`Hello world`显示在主窗口中

# Step3. 函数封装

In [2]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel
from PyQt5.QtCore import QCoreApplication

def app_init():
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    return app

def app_exit(app):
    app.exec()
    
def win_init():
    w = QWidget()
    w.setWindowTitle("Hello")
    w.show()
    return w

def text_init(w):
    text = QLabel()
    text.setText("Hello World")
    text.setParent(w)
    text.show()

def main():
    app = app_init()
    w = win_init()
    text_init(w)
    app_exit(app)
    
if __name__ == '__main__':
    main()

使用def关键字完成函数的封装。并根据功能不通拆分为多个子函数

# Step4. 显示时间-类的使用

In [1]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-

import sys
# from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QDialog
# from PyQt5.QtCore import QCoreApplication, QTimer, QDateTime

### 导入所有模块
from PyQt5 import QtGui, QtCore, QtWidgets
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *

In [2]:
class showTime(QDialog):
    def __init__(self):
        super(showTime, self).__init__()
        self.resize(500, 400)
        self.setWindowTitle("Rtc Time")
        self.label = QLabel(self)
        self.label.setFixedWidth(300)
        self.label.move(100, 80)
        self.label.setStyleSheet("QLabel{background:white;}" \
                                 "QLabel{color:rgb(300,300,300,120);"\
                                 "font-size:15px;font-weight:bold;font-family:宋体;}" \
                                 )
        # 动态显示时间在label上
        timer = QTimer(self)
        timer.timeout.connect(self.showtime)
        timer.start()
        
    def showtime(self):
        datetime = QDateTime.currentDateTime()
        text = datetime.toString()
        self.label.setText(5*" " + text)


* `class showTime(QDialog):` 定义了showTime类，此类继承自类QDialog。  
* `def __init__(self):` 定义了函数`__init__`，此函数在执行类的定义时自动被执行
    * `super` 函数用于调用父类（超类）的一个方法/函数，super是用于解决多重继承问题的。此处的操作等价与调用了QDialog的__init__方法，等于创建了1个QDialog窗口
    * `self.resize`和`self.setWindowTitle`分别用于修改QDialog窗口的大小和title。
    * `self.label = QLabel(self)`此代码定义了一个label并且此label的父类为self即之前定义的QDialog窗口
    * `setFixedWidth`、`move`、`setStyleSheet`三个函数用于修改label的尺寸、位置、显示风格属性
    * `QTimer`类提供了定时器，用于定时循环执行特定函数，定时器默认每秒中断循环一次，定时器溢出时调用函数showtime，此函数通过定时器的timeout的connect方法完成配置`timer.timeout.connect(self.showtime)`, `timer.start()`启动定时器。此处实际上使用的是信号与槽机制，后续将专门讲解
* `def show time(self):`此函数被定时调用
    * `QDateTime`类提供了`currentDateTime`方法可获取系统当前时间。
    * `datetime.toString()`将日期对象转变为字符串
    * `self.label.setText`用于修改label的Text属性，实现更新时间的目的


In [3]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    my = showTime()
    my.show()
    app.exec_()

## C++版本

```C++

```

